In [1]:
from selenium import webdriver
import time
import pandas as pd
import re
from selenium.webdriver.common.by import By
#from datetime import datetime, date, timedelta
from selenium.webdriver.chrome.options import Options
import requests
import json

from webdriver_manager.chrome import ChromeDriverManager
import mysql
import mysql.connector
import os

import datetime
import unicodedata

from dotenv import load_dotenv
load_dotenv(".env")

True

In [37]:
import sshtunnel
from sshtunnel import SSHTunnelForwarder

server = sshtunnel.SSHTunnelForwarder((os.getenv('SSH_USERNAME'), 10022), 
    ssh_username="pachislot777", 
    ssh_private_key_password=os.getenv('SSH_PRIVATE_KEY_PASSWORD'), 
    ssh_pkey="sercret/akasaka.key", 
    remote_bind_address=("mysql8055.xserver.jp", 3306 )) 
# SSH接続確認


# 現在時刻
today = datetime.datetime.now()
print(today)
#[結果] 2021-08-23 07:12:20.806648
# 1日後
today_str:str = today.strftime('%Y-%m-%d')
eight_days_after:str = (today + datetime.timedelta(days=8)).strftime('%Y-%m-%d')
yesterday:str = (today + datetime.timedelta(days=-2)).strftime('%Y-%m-%d')
#### Create dataframe from resultant table ####

server.start()

print(f"local bind port: {server.local_bind_port}")
# データベース接続
cnx = mysql.connector.connect(
    host="localhost", 
    port=server.local_bind_port, 
    user=os.getenv('WORDPRESS_DB_ID'), 
    password=os.getenv('DB_PASSWORD'), 
    database=os.getenv('WORDPRESS_DB_NAME'), 
    charset='utf8',
    use_pure=True
    )

# 接続確認
print(f"sql connection status: {cnx.is_connected()}")
cursor = cnx.cursor()

2023-06-27 02:28:46.805637
local bind port: 49718
sql connection status: True


### テーブルの数を数える

In [41]:
cursor.execute("show tables;")
myresult = cursor.fetchall()
 
for x in myresult:
  print(x)

('maptable',)
('parlar_data',)
('pledge',)
('schedule',)


In [34]:
sql = f"""
        SELECT *
        FROM pledge
        """
print(sql)
cursor.execute(sql)
#cols = [col[0] for col in cursor.description]
sql_syuzai_report_all_df = pd.DataFrame(cursor.fetchall(),columns = ['id','取材名','媒体名','公約内容','取得時間'])
sql_syuzai_report_all_df


        SELECT *
        FROM pledge
        


,id,取材名,媒体名,公約内容,取得時間
0,1,ｙｔｒ来店実践,SEVENSTV,来店実践,2023-03-16 03:56:31
1,2,Ａレポート,アツ姫,Aタイプに高設定投入,2023-03-16 03:56:31
2,3,黒バラの極意(来店ver),ジャンバリ.TV,,2023-03-16 03:56:31
3,4,鮮魚レポート,アメスロ,,2023-03-16 03:56:31
4,5,魔法少女レポート スロット取材,爆ガチ！,まどかマギカシリーズが狙い目,2023-03-16 03:56:31
...,...,...,...,...,...
2126,101375,しおねえ来店,エースプロ,None,2023-06-17 09:30:52
2127,101376,星のパチは友を呼ぶ,パチとスロは友を呼ぶ,None,2023-06-18 09:25:03
2128,101377,シーサ。の法則,P-martTV,None,2023-06-24 01:48:28
2129,101378,ジョージ来店,その他取材・企画,None,2023-06-24 01:48:28


In [35]:
print(cursor.execute(sql))

None
